<a href="https://colab.research.google.com/github/ILYASbz/Speech-Recognition/blob/main/Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/openai/whisper.git -q


In [ ]:
! pip install gradio -q


In [ ]:
!pip install gTTS


In [ ]:
!pip install bardapi

In [ ]:
import gradio as gr
import whisper
from gtts import gTTS
import IPython.display as ipd
from bardapi import Bard
import os
import re

model = whisper.load_model("base")

def extract_arabic_text(paragraph):
    lines = paragraph.split('\n')
    arabic_texts = []
    for line in lines:
        arabic_text = re.findall(r'[\u0600-\u06FF]+', line)
        if arabic_text:
            arabic_texts.append(' '.join(arabic_text))
    return arabic_texts




def transcribe(audio):
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions(fp16=False)
    result = whisper.decode(model, mel, options)

    # Join the elements of the result list to make it a single string
    text = "".join(result.text)
    print(text)
    os.environ["_BARD_API_KEY"] = "YOUR_KEY"
    ar = f"translate \" '{text}' \" to darija moroccan with arabic letters "
    at = Bard().get_answer(str(ar))['content']
    generated_text2 = extract_arabic_text(at)
    # Convert the list back to a string for gTTS
    generated_text2 = generated_text2[0]
    tts = gTTS(text=generated_text2, lang="ar")
    tts.save("darija_speech.mp3")
    ipd.display(ipd.Audio("darija_speech.mp3"))
    return generated_text2

gr.Interface(
    title='speech english to darija voice :) <3 ',
    description="GALT LIK DIHA L WAD",
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath")
    ],
    outputs=[
        "textbox"
    ],
    live=True).launch(debug=True)
